# ENGIE QTEM Data Challenge – 2025  

This notebook is a comprehensive and reproducible workflow to solve the ENGIE QTEM Data Challenge. The challenge is focused on optimizing a renewable energy asset portfolio based on weather, production, and market pricing data.

### Key Business Questions:
- Which combination of renewable energy assets yields the highest production with the least variability?
- How do revenue considerations (market price data) affect the optimal portfolio selection?
- What are the differences between production-driven and revenue-driven optimization?
- How would the results change if real-world inefficiencies (e.g. curtailments) were removed?

---

### Final Deliverables:
- Reproducible optimization models (production & revenue)
- Portfolio performance metrics (mean output, standard deviation, volatility)
- Comparative analysis & sensitivity insights
- Final report and presentation-ready visualizations

---

### Data Overview:
- **Solar Production Sites:** 71 (Belgium, Germany, Netherlands)
- **Wind Sites:** 99 (onshore + offshore)
- **Weather Variables:** Temperature, wind speed/direction, cloud cover, solar radiation
- **Prices:** Quarter-hourly and hourly market prices (Day-Ahead, Intraday, ISP)
- **Metadata:** Site locations, variable definitions

---

## Available Data:

- **Solar Data Folder:** Contains files for each solar site:
  - Dew Point (`d2m`)
  - Total Cloud Cover (`tcc`)
  - Temperature (`t2m`)
  - Solar Radiation (`ssr`)
  - Wind Angle at 10m (`angle10`)
  - Wind Speed at 10m (`speed10`)
  - Load Factor (`factor`)

- **Wind Data Folder:** Contains files for each wind site:
  - Wind Angle at 100m (`angle100`)
  - Wind Speed at 100m (`speed100`)
  - Load Factor (`factor`)

- **Price and Liquidity Data:**
  - Day Ahead and Intraday prices.

- **Additional Files:**
  - Sites anonymized data (`sites_anonymized.csv`)
  - Data Dictionary (`data_dictionary.xlsx`)

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from glob import glob
from tqdm import tqdm
import os

# 1. Comprehensive Data Assessment

In this section, we:
- Explore metadata to understand the site distribution
- Examine weather, production, and price data structure
- Identify missing values, timestamp irregularities, and data format issues
- Prepare for integration and cleaning

### 📊 Price & Liquidity Data Cleaning Summary

We started with a raw pricing dataset containing 59 columns and 132,310 timestamped records, covering multiple market types (Day-Ahead, Intraday, Imbalance) across Belgium (BE), Germany (DE), the Netherlands (NL), and France (FR).

#### ✅ Actions Taken:
- Removed all `volume` columns, which are not required in the ENGIE optimization objective (no use for liquidity or transaction volume).
- Dropped all columns related to France (`FR`) since the challenge scope is limited to BE, DE, and NL.
- Excluded rarely populated or redundant formats such as half-hour (HH) pricing and kept only **Hourly and Quarter-Hourly (QH)** series where reasonably populated.
- Retained all Day-Ahead (`DA_*`) pricing columns.
- Retained relevant Intraday pricing layers: `ID1`, `ID3`, and `IDFull`, for both `Hourly` and `QH` formats.
- Retained flat imbalance prices `ISP_*`, as well as `ISP_SHORT_NL` and `ISP_LONG_NL` for modeling imbalance market logic in Part 2 or Part 4.

#### 📦 Resulting Dataset:
- Final shape: **132,310 rows × 26 columns**
- Includes all pricing columns required for Parts **2, 3, and 4** of the challenge.
- Structured to support reproducible modeling and easy merging with production and weather data.

In [3]:
# Load metadata
sites_df = pd.read_csv("/Users/hossameldinelhendawy/Documents/QDC-Lib/sites_anonymized.csv")
price_df = pd.read_csv("/Users/hossameldinelhendawy/Documents/QDC-Lib/intraday_indices_prices_and_liquidity.csv")

# Datetime column modification
price_df.rename(columns={price_df.columns[0]: "datetime"}, inplace=True)
price_df['datetime'] = pd.to_datetime(price_df['datetime'])

/var/folders/5f/8x4b66f12jxblmgjlz1qymv80000gn/T/ipykernel_1871/3748134077.py:7: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  price_df['datetime'] = pd.to_datetime(price_df['datetime'])


In [4]:
# Selection of relevant price columns for Parts 1–4
columns_to_keep = ['datetime']

# Add DA prices
for country in ['BE', 'DE', 'NL']:
    col = f'DA_{country}'
    if col in price_df.columns:
        columns_to_keep.append(col)

# Add hourly and QH intraday prices
intraday_types = ['ID1', 'ID3', 'IDFull']
resolutions = ['Hourly', 'QH']

for market in intraday_types:
    for resolution in resolutions:
        for country in ['BE', 'DE', 'NL']:
            col = f'{market}_{resolution}_{country}_price'
            if col in price_df.columns:
                columns_to_keep.append(col)

# Add Imbalance prices
for country in ['BE', 'DE', 'NL']:
    # Flat ISP
    col_flat = f'ISP_{country}'
    if col_flat in price_df.columns:
        columns_to_keep.append(col_flat)
    # NL-specific imbalance split
    col_short = f'ISP_SHORT_{country}'
    col_long = f'ISP_LONG_{country}'
    if col_short in price_df.columns:
        columns_to_keep.append(col_short)
    if col_long in price_df.columns:
        columns_to_keep.append(col_long)

# Apply filter
price_cleaned_df = price_df[columns_to_keep].copy()

## ☀️ Solar Data Preprocessing Summary

We prepared the solar dataset for modeling by executing the following structured pipeline:

### 1. Site Discovery & Variable Definition
- Loaded all `.csv` files from the solar folder
- Extracted 71 unique site IDs
- Defined the target variables:  
  - Weather: `d2m`, `tcc`, `t2m`, `ssr`, `angle10`, `speed10`  
  - Production: `load_factor`

### 2. Per-Site Loading & Merging
- For each site, we loaded its 7 variable files
- Parsed and standardized the `datetime` column
- Merged all variables on `datetime`
- Added a `site_id` column
- Stored the result in `solar_site_data[site_id]`

### 3. Timeline Construction
- Extracted all timestamps across all sites
- Built a unified hourly index:
  - Start: `2018-12-31 23:00:00`
  - End: `2024-10-01 00:00:00`
  - Total timestamps: `50,402`
  - Frequency: **Hourly**

### 4. Site Reindexing
- Reindexed every site’s dataframe to the unified hourly timeline
- Preserved missing data as `NaN`
- Stored aligned sites in `solar_site_data_aligned`

### 5. Final Stacking
- Combined all aligned sites into a single dataframe `solar_all_df`
- Shape: `3,578,542 rows × 9 columns`
- Columns: `datetime`, 6 weather vars, `load_factor`, `site_id`
- Verified 71 unique sites and 50,402 hourly timestamps

This cleaned dataset is now ready for merging with price data and optimization in Phases 2 and 3.


In [19]:
# Setup path
solar_path = "/Users/hossameldinelhendawy/Documents/QDC-Lib/solar"

# All solar files
solar_files = glob(os.path.join(solar_path,"*.csv"))

# Extract site IDs from filenames
solar_site_ids = sorted(
    list(set([f.split('_')[-1].replace('.csv', '') for f in solar_files]))
)

In [20]:
# Variables per site
solar_vars = ['d2m', 'tcc', 't2m', 'ssr', 'angle10', 'speed10', 'load_factor']

def load_solar_site_data(site_id, folder_path):
    dfs = []
    
    for var in solar_vars:
        if var == 'load_factor':
            file_name = f'load_factor_{site_id}.csv'
        else:
            file_name = f'era5_{var}_{site_id}.csv'
        
        file_path = os.path.join(folder_path, file_name)
        
        if not os.path.exists(file_path):
            print(f"[WARNING] Missing file for {var} at site {site_id}")
            continue
        
        df = pd.read_csv(file_path)
        
        # Rename datetime column and parse
        df.rename(columns={df.columns[0]: 'datetime'}, inplace=True)
        df['datetime'] = pd.to_datetime(df['datetime'])
        
        # Rename value column to the variable name
        df.rename(columns={df.columns[1]: var}, inplace=True)
        dfs.append(df)
    
    if not dfs:
        print(f"[ERROR] No data found for site {site_id}")
        return None
    
    # Merge on datetime
    merged_df = dfs[0]
    for df in dfs[1:]:
        merged_df = pd.merge(merged_df, df, on='datetime', how='outer')
    
    merged_df['site_id'] = site_id
    return merged_df

In [21]:
# Extract solar sites files into one dict
solar_site_data = {}

for site_id in solar_site_ids:
    try:
        merged_site_df = load_solar_site_data(site_id, solar_path)
        if merged_site_df is not None:
            solar_site_data[site_id] = merged_site_df
    except Exception as e:
        print(f"[ERROR] Failed to load site {site_id}: {e}")

In [28]:
# How many wind sites loaded successfully?
print("✅ Solar sites loaded:", len(solar_site_data))

# Print column names for a sample site
sample_id = list(solar_site_data.keys())[0]
print("📋 Sample columns for site", sample_id)
print(solar_site_data[sample_id].columns.tolist())

✅ Solar sites loaded: 71
📋 Sample columns for site 026396
['datetime', 'd2m', 'tcc', 't2m', 'ssr', 'angle10', 'speed10', 'load_factor', 'site_id']


In [22]:
# Collect all datetime values from all sites
datetime_series_list = [df['datetime'] for df in solar_site_data.values()]
all_timestamps = pd.concat(datetime_series_list, ignore_index=True)

# Drop duplicates and sort
all_timestamps = pd.to_datetime(all_timestamps.unique())
all_timestamps = pd.Series(sorted(all_timestamps))

# Inspect time coverage and frequency
print("📆 Datetime coverage:")
print("Start:", all_timestamps.min())
print("End  :", all_timestamps.max())
print("\n🕒 Time interval frequencies:")
print(all_timestamps.diff().value_counts().head())
print("\n🔢 Total unique timestamps:", len(all_timestamps))

📆 Datetime coverage:
Start: 2018-12-31 23:00:00+00:00
End  : 2024-10-01 00:00:00+00:00

🕒 Time interval frequencies:
0 days 01:00:00    50401
Name: count, dtype: int64

🔢 Total unique timestamps: 50402


In [23]:
# Align each site to the master datetime index

# Create the master datetime index (hourly)
datetime_index = pd.DatetimeIndex(all_timestamps)

# New dictionary to hold reindexed data
solar_site_data_aligned = {}

# Use tqdm to track loading progress
for site_id, df in tqdm(solar_site_data.items(), desc="Reindexing sites"):
    df = df.set_index('datetime')
    
    # Reindex to fill in all missing timestamps with NaNs
    df = df.reindex(datetime_index)
    
    # Keep the site_id in a column
    df['site_id'] = site_id
    
    # Store in the new aligned dictionary
    df.index.name = 'datetime'
    solar_site_data_aligned[site_id] = df.reset_index()


Reindexing sites: 100%|██████████| 71/71 [00:00<00:00, 84.83it/s]


In [24]:
# Pick a sample site to check
sample_id = list(solar_site_data_aligned.keys())[0]
sample_df = solar_site_data_aligned[sample_id]

print(f"✅ Sample site: {sample_id}")
print("Rows:", sample_df.shape[0])
print("Expected rows:", len(datetime_index))
display(sample_df.head())

✅ Sample site: 026396
Rows: 50402
Expected rows: 50402


,datetime,d2m,tcc,t2m,ssr,angle10,speed10,load_factor,site_id
0,2018-12-31 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,026396
1,2019-01-01 00:00:00+00:00,279.732910,1.000000,281.158844,0.0,0.179575,3.395970,0.000084,026396
2,2019-01-01 01:00:00+00:00,279.169403,0.941495,280.837677,0.0,0.228696,3.402579,0.000084,026396
3,2019-01-01 02:00:00+00:00,278.669128,0.998993,280.659973,0.0,0.226669,3.528860,0.000085,026396
4,2019-01-01 03:00:00+00:00,278.274231,0.990234,280.580109,0.0,0.284809,3.603586,0.000084,026396


In [25]:
# Combine all aligned sites into one big dataframe
solar_all_df = pd.concat(solar_site_data_aligned.values(), ignore_index=True)

print("✅ Combined solar dataset shape:", solar_all_df.shape)
print("🔍 Columns:", solar_all_df.columns.tolist())

# Check number of unique sites and timestamps
print("📌 Unique site IDs:", solar_all_df['site_id'].nunique())
print("📆 Unique timestamps:", solar_all_df['datetime'].nunique())

# Optional: quick data preview
display(solar_all_df.head())


✅ Combined solar dataset shape: (3578542, 9)
🔍 Columns: ['datetime', 'd2m', 'tcc', 't2m', 'ssr', 'angle10', 'speed10', 'load_factor', 'site_id']
📌 Unique site IDs: 71
📆 Unique timestamps: 50402


,datetime,d2m,tcc,t2m,ssr,angle10,speed10,load_factor,site_id
0,2018-12-31 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,026396
1,2019-01-01 00:00:00+00:00,279.732910,1.000000,281.158844,0.0,0.179575,3.395970,0.000084,026396
2,2019-01-01 01:00:00+00:00,279.169403,0.941495,280.837677,0.0,0.228696,3.402579,0.000084,026396
3,2019-01-01 02:00:00+00:00,278.669128,0.998993,280.659973,0.0,0.226669,3.528860,0.000085,026396
4,2019-01-01 03:00:00+00:00,278.274231,0.990234,280.580109,0.0,0.284809,3.603586,0.000084,026396


## 🌬️ Wind Data Preprocessing Summary

We applied a structured pipeline to prepare the wind site data for portfolio optimization. This followed the same logic as the solar data, with minor adjustments based on available variables.

---

### 1. Site Discovery & Variable Definition
- Loaded all `.csv` files from the `wind/` folder
- Extracted **99 unique wind site IDs**
- Defined 3 target variables:
  - `angle100` – wind direction at 100m
  - `speed100` – wind speed at 100m
  - `load_factor` – actual site production as a fraction of capacity

---

### 2. Per-Site Loading & Merging
- Loaded all available variable files for each wind site
- Parsed and standardized the `datetime` column
- Renamed value columns for clarity
- Merged the variables on `datetime`
- Added a `site_id` column
- Stored merged site data in `wind_site_data[site_id]`

Missing files (e.g., `angle100`) were gracefully skipped with warnings. Sites with no valid files were excluded.

---

### 3. Reindexing to Master Timeline
- Reused the unified hourly `datetime_index` created from the solar phase
- Reindexed all wind sites to this common timeline:
  - Start: `2018-12-31 23:00:00`
  - End: `2024-10-01 00:00:00`
  - Total timestamps: `50,402`
- Preserved `NaN`s for any missing intervals
- Stored aligned dataframes in `wind_site_data_aligned`

---

### 4. Final Stacking
- Concatenated all aligned wind sites into a single long-format dataframe `wind_all_df`
- Shape: `4,989,798 rows × 5 columns`
- Columns: `datetime`, `angle100`, `speed100`, `load_factor`, `site_id`
- Verified:
  - ✅ 99 unique `site_id`s
  - ✅ 50,402 unique hourly `datetime`s

---

### ✅ Outcome
The `wind_all_df` dataframe is now fully cleaned and aligned, ready to be merged with solar data and price data for Phase 2:
- Production analysis
- Portfolio optimization
- Revenue modeling

In [26]:
# Setup path
wind_path = "/Users/hossameldinelhendawy/Documents/QDC-Lib/wind"

# All wind files
wind_files = glob(os.path.join(wind_path, "*.csv"))

# Extract wind site IDs
wind_site_ids = sorted(
    list(set([f.split('_')[-1].replace('.csv', '') for f in wind_files]))
)

# Variables per wind site
wind_vars = ['angle100', 'speed100', 'load_factor']

def load_wind_site_data(site_id, folder_path):
    dfs = []
    
    for var in wind_vars:
        if var == 'load_factor':
            file_name = f'load_factor_{site_id}.csv'
        else:
            file_name = f'era5_{var}_{site_id}.csv'
        
        file_path = os.path.join(folder_path, file_name)
        
        if not os.path.exists(file_path):
            print(f"[WARNING] Missing file for {var} at site {site_id}")
            continue
        
        df = pd.read_csv(file_path)
        
        # Rename datetime column and parse
        df.rename(columns={df.columns[0]: 'datetime'}, inplace=True)
        df['datetime'] = pd.to_datetime(df['datetime'])
        
        # Rename value column to the variable name
        df.rename(columns={df.columns[1]: var}, inplace=True)
        dfs.append(df)
    
    if not dfs:
        print(f"[ERROR] No data found for site {site_id}")
        return None
    
    # Merge on datetime
    merged_df = dfs[0]
    for df in dfs[1:]:
        merged_df = pd.merge(merged_df, df, on='datetime', how='outer')
    
    merged_df['site_id'] = site_id
    return merged_df

# Load all wind sites into dictionary
wind_site_data = {}

for site_id in wind_site_ids:
    try:
        merged_site_df = load_wind_site_data(site_id, wind_path)
        if merged_site_df is not None:
            wind_site_data[site_id] = merged_site_df
    except Exception as e:
        print(f"[ERROR] Failed to load site {site_id}: {e}")


[WARNING] Missing file for angle100 at site 980877


In [ ]:
# How many wind sites loaded successfully?
print("✅ Wind sites loaded:", len(wind_site_data))

# Print column names for a sample site
sample_id = list(wind_site_data.keys())[0]
print("📋 Sample columns for site", sample_id)
print(wind_site_data[sample_id].columns.tolist())

✅ Wind sites loaded: 99
📋 Sample columns for site 008807
['datetime', 'angle100', 'speed100', 'load_factor', 'site_id']


In [29]:
# Master index from solar
datetime_index = pd.DatetimeIndex(all_timestamps)

# New dictionary for aligned wind data
wind_site_data_aligned = {}

for site_id, df in tqdm(wind_site_data.items(), desc="Reindexing wind sites"):
    df = df.set_index('datetime')
    df = df.reindex(datetime_index)
    df['site_id'] = site_id
    df.index.name = 'datetime'
    wind_site_data_aligned[site_id] = df.reset_index()

Reindexing wind sites: 100%|██████████| 99/99 [00:00<00:00, 104.95it/s]


In [ ]:
# Pick a sample site to check
sample_id = list(wind_site_data_aligned.keys())[0]
sample_df = wind_site_data_aligned[sample_id]

print(f"✅ Sample wind site: {sample_id}")
print("Rows:", sample_df.shape[0])
print("Expected rows (from master index):", len(datetime_index))
print("Missing values per column:")
print(sample_df.isnull().sum())

display(sample_df.head())

✅ Sample wind site: 008807
Rows: 50402
Expected rows (from master index): 50402
Missing values per column:
datetime           0
angle100         337
speed100         337
load_factor    32858
site_id            0
dtype: int64


,datetime,angle100,speed100,load_factor,site_id
0,2018-12-31 23:00:00+00:00,NaN,NaN,NaN,008807
1,2019-01-01 00:00:00+00:00,0.219218,7.563506,NaN,008807
2,2019-01-01 01:00:00+00:00,0.295403,7.909504,NaN,008807
3,2019-01-01 02:00:00+00:00,0.276935,8.721675,NaN,008807
4,2019-01-01 03:00:00+00:00,0.265117,9.621751,NaN,008807


In [31]:
# Combine all aligned wind sites into one big dataframe
wind_all_df = pd.concat(wind_site_data_aligned.values(), ignore_index=True)

print("✅ Combined wind dataset shape:", wind_all_df.shape)
print("🔍 Columns:", wind_all_df.columns.tolist())

# Check number of unique sites and timestamps
print("📌 Unique site IDs:", wind_all_df['site_id'].nunique())
print("📆 Unique timestamps:", wind_all_df['datetime'].nunique())

# Optional: preview
display(wind_all_df.head())

✅ Combined wind dataset shape: (4989798, 5)
🔍 Columns: ['datetime', 'angle100', 'speed100', 'load_factor', 'site_id']
📌 Unique site IDs: 99
📆 Unique timestamps: 50402


,datetime,angle100,speed100,load_factor,site_id
0,2018-12-31 23:00:00+00:00,NaN,NaN,NaN,008807
1,2019-01-01 00:00:00+00:00,0.219218,7.563506,NaN,008807
2,2019-01-01 01:00:00+00:00,0.295403,7.909504,NaN,008807
3,2019-01-01 02:00:00+00:00,0.276935,8.721675,NaN,008807
4,2019-01-01 03:00:00+00:00,0.265117,9.621751,NaN,008807


In [ ]:
# Add source tag
solar_all_df['source'] = 'solar'
wind_all_df['source'] = 'wind'

# Ensure site_id columns are integers for join
solar_all_df['site_id'] = solar_all_df['site_id'].astype(int)
wind_all_df['site_id'] = wind_all_df['site_id'].astype(int)
sites_df['site'] = sites_df['site'].astype(int)

# Merge solar and wind into one full production dataframe
production_df = pd.concat([solar_all_df, wind_all_df], ignore_index=True)

# Join with metadata
all_sites_df = production_df.merge(
    sites_df.rename(columns={'site': 'site_id'}),
    on='site_id',
    how='left'
)

# Check results
print("✅ Final unified production dataset shape:", all_sites_df.shape)
print("🔍 Columns:", all_sites_df.columns.tolist())
print("📌 Unique sites:", all_sites_df['site_id'].nunique())
print("🕒 Unique timestamps:", all_sites_df['datetime'].nunique())
print("🔢 Source breakdown:")
print(all_sites_df['source'].value_counts())

# Preview
display(all_sites_df.head())

✅ Final unified production dataset shape: (9072360, 16)
🔍 Columns: ['datetime', 'd2m', 'tcc', 't2m', 'ssr', 'angle10', 'speed10', 'load_factor', 'site_id', 'source', 'angle100', 'speed100', 'latitude', 'longitude', 'type', 'capacity']
📌 Unique sites: 170
🕒 Unique timestamps: 50402
🔢 Source breakdown:
source
wind     4989798
solar    4082562
Name: count, dtype: int64


,datetime,d2m,tcc,t2m,ssr,angle10,speed10,load_factor,site_id,source,angle100,speed100,latitude,longitude,type,capacity
0,2018-12-31 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26396,solar,NaN,NaN,51.091425,3.780883,Solar,448.0
1,2019-01-01 00:00:00+00:00,279.732910,1.000000,281.158844,0.0,0.179575,3.395970,0.000084,26396,solar,NaN,NaN,51.091425,3.780883,Solar,448.0
2,2019-01-01 01:00:00+00:00,279.169403,0.941495,280.837677,0.0,0.228696,3.402579,0.000084,26396,solar,NaN,NaN,51.091425,3.780883,Solar,448.0
3,2019-01-01 02:00:00+00:00,278.669128,0.998993,280.659973,0.0,0.226669,3.528860,0.000085,26396,solar,NaN,NaN,51.091425,3.780883,Solar,448.0
4,2019-01-01 03:00:00+00:00,278.274231,0.990234,280.580109,0.0,0.284809,3.603586,0.000084,26396,solar,NaN,NaN,51.091425,3.780883,Solar,448.0


In [ ]:
# Remove timezone from price_cleaned_df datetime
price_cleaned_df['datetime'] = pd.to_datetime(price_cleaned_df['datetime'], utc=True)
price_cleaned_df['datetime'] = price_cleaned_df['datetime'].dt.tz_localize(None)

# Remove timezone from all_sites_df datetime
all_sites_df['datetime'] = pd.to_datetime(all_sites_df['datetime'], utc=True)
all_sites_df['datetime'] = all_sites_df['datetime'].dt.tz_localize(None)

# ✅ Optional check
assert all_sites_df['datetime'].dt.tz is None
assert price_cleaned_df['datetime'].dt.tz is None

# Merge
exploratory_df = all_sites_df.merge(price_cleaned_df, on='datetime', how='left')

# Sanity check
print("✅ Exploratory dataset shape:", exploratory_df.shape)
print("📌 Unique sites:", exploratory_df['site_id'].nunique())
print("🕒 Unique timestamps:", exploratory_df['datetime'].nunique())
print("🔢 Sources:", exploratory_df['source'].value_counts())
display(exploratory_df.head())


✅ Exploratory dataset shape: (9072360, 41)
📌 Unique sites: 170
🕒 Unique timestamps: 50402
🔢 Sources: source
wind     4989798
solar    4082562
Name: count, dtype: int64


,datetime,d2m,tcc,t2m,ssr,angle10,speed10,load_factor,site_id,source,...,IDFull_Hourly_BE_price,IDFull_Hourly_DE_price,IDFull_Hourly_NL_price,IDFull_QH_BE_price,IDFull_QH_DE_price,IDFull_QH_NL_price,ISP_BE,ISP_DE,ISP_SHORT_NL,ISP_LONG_NL
0,2018-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26396,solar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01 00:00:00,279.732910,1.000000,281.158844,0.0,0.179575,3.395970,0.000084,26396,solar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01 01:00:00,279.169403,0.941495,280.837677,0.0,0.228696,3.402579,0.000084,26396,solar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01 02:00:00,278.669128,0.998993,280.659973,0.0,0.226669,3.528860,0.000085,26396,solar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01 03:00:00,278.274231,0.990234,280.580109,0.0,0.284809,3.603586,0.000084,26396,solar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Only production-related variables
part1_df = exploratory_df[[
    'datetime',
    'site_id',
    'load_factor',
    'source',
    'type',
    'latitude',
    'longitude'
]].copy()

# Optional: drop rows with missing load factor
part1_df = part1_df[~part1_df['load_factor'].isna()]

# Check result
print("✅ Phase 1 dataset shape:", part1_df.shape)
print("📌 Unique sites:", part1_df['site_id'].nunique())
print("🕒 Unique timestamps:", part1_df['datetime'].nunique())
print("🔢 Source breakdown:")
print(part1_df['source'].value_counts())
display(part1_df.head())

✅ Phase 1 dataset shape: (4677296, 7)
📌 Unique sites: 170
🕒 Unique timestamps: 50401
🔢 Source breakdown:
source
wind     2584776
solar    2092520
Name: count, dtype: int64


,datetime,site_id,load_factor,source,type,latitude,longitude
1,2019-01-01 00:00:00,26396,0.000084,solar,Solar,51.091425,3.780883
2,2019-01-01 01:00:00,26396,0.000084,solar,Solar,51.091425,3.780883
3,2019-01-01 02:00:00,26396,0.000085,solar,Solar,51.091425,3.780883
4,2019-01-01 03:00:00,26396,0.000084,solar,Solar,51.091425,3.780883
5,2019-01-01 04:00:00,26396,0.000085,solar,Solar,51.091425,3.780883
